In [ ]:
import pandas as pd
import sys
import os
import os.path
from pathlib import Path
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from scipy.stats import pearsonr
#import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
import joblib
import json
import sys
import math
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
os.chdir(Path(__file__).parents[1])
sys.path.append(Path(__file__).parents[1])

run = "run1"

In [ ]:
# use one parameter dict
if len(sys.argv)!=2:
    argument = ['-',1]
else:
    print('cmd entry:', sys.argv)
    argument = sys.argv
which_run = int(argument[1]) - 1
print("emtpb: Parallel run "+str(which_run))

### costum
which_run = 24
### costum

EMTscores = pd.read_csv("metadata/EMTscores.csv")
cancertypes = np.concatenate((['PANCAN'], EMTscores['TCGA Desc'].unique()))
preds_dir = "metadata/"+run+"/predictions/"+cancertypes[which_run]+"/"
model_dir = "metadata/"+run+"/models/"+cancertypes[which_run]+"/"
os.makedirs(preds_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

mut = pd.read_csv("metadata/matrix_mut_"+cancertypes[which_run]+".csv")
resp_full = pd.read_csv("metadata/matrix_resp.csv")
cols = resp_full.columns[resp_full.columns.str.contains('GDSC')]

In [ ]:
# Functions
def run_elasticnet_cv(X, y, outer_seed, inner_seed, model_dir, preds_dir, names, which_index, outer_folds = 5, inner_folds = 5, repeats = 5, baseline = False, save_models = False):
    
    # predictions save path and check if its exists already
    preds_filename = os.path.join(preds_dir, f"predictions_{which_index}_{baseline}.csv")
    if Path(preds_filename).exists():
        print("emtpb: iteration "+str(index)+" already exists... skipping...")
        return
    
    # remove interesting variable for baseline
    if baseline:
        X = np.delete(X, 0, axis=1)
    
    # initialize list of all preds
    global all_all_predictions
    all_all_predictions = []
    all_all_truth = []
    names = pd.DataFrame(names)
    names = pd.concat([names]*repeats, ignore_index = True)
    
    # initialize the outer cross-validation
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=outer_seed)

    # initialize the inner cross-validation
    inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=inner_seed)

        # repeat the outer cross-validation 5 times
    for repeat in range(repeats):
        
        # initialize the list to store the predictions
        all_predictions = []
        all_truth = []

        # set the seed for the outer cross-validation
        outer_cv.random_state = outer_seed + repeat

        # loop through the splits of the outer cross-validation
        for i, (train_index, test_index) in enumerate(outer_cv.split(X)):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # initialize the ElasticNetCV model
            model = ElasticNetCV(l1_ratio=[0.01, .1, .5, .9, .95, 1],
                                 cv=inner_cv,
                                 random_state=inner_seed)

            # fit the model on the training data
            model.fit(X_train, y_train)

            # make predictions on the test set
            y_pred = model.predict(X_test)

            # save the predictions
            all_predictions.append(y_pred)
            all_truth.append(y_test)

            # save the model
            model_filename = os.path.join(model_dir, f"model_{which_index}_{baseline}_{repeat}_{i}.joblib")
            if save_models:
                joblib.dump(model, model_filename)

        # concatenate all the predictions from cv
        all_predictions = np.concatenate(all_predictions)
        all_truth = np.concatenate(all_truth)
        
        # save the cv results and append to resampling
        all_all_predictions.append(all_predictions)
        all_all_truth.append(all_truth)
    
    # concatenate all the predictions
    all_all_predictions = np.concatenate(all_all_predictions)
    all_all_truth = np.concatenate(all_all_truth)
    
    names["preds"] = all_all_predictions
    names["truth"] = all_all_truth#np.concatenate([y]*repeats)
    names["repeat"] = np.concatenate([[i]*len(y) for i in range(repeats)])
    
    # save predictions 
    names.to_csv(preds_filename, index=False)


In [ ]:
# benchmark 
verbose = True
example = False # "1159-GDSC2"

for index in range(len(cols)):
    
    if not not example:
        index = np.where(cols == "1559-GDSC2")[0][0]
    
    print("emtpb: modelling iteration "+str(index)+"...")
    #index = np.where(cols == "1559-GDSC2")[0][0]

    # get one response column
    resp = resp_full.copy()
    resp = resp[["COSMIC ID","TCGA Desc",cols[index]]]
    mat = pd.merge(EMTscores, resp, how='outer')
    if cancertypes[which_run] != "PANCAN":
        mat = mat[mat['TCGA Desc'] == cancertypes[which_run]]
    mat = pd.merge(mat, mut, how='outer')
    mat = pd.get_dummies(mat)
    mat_df = mat
    print("empb: shape before removing nan is "+str(mat.shape))
    mat.dropna(inplace=True)
    print("empb: shape after removing nan is "+str(mat.shape))
    names = mat['COSMIC ID']
    mat.drop(columns=['COSMIC ID'], inplace=True)
    response = mat.iloc[:,1]
    mat = mat.drop(mat.columns[1], axis=1)
    scale = StandardScaler()
    mat = scale.fit_transform(mat)

    for baseline in [True, False]:
        run_elasticnet_cv(
            X = mat,
            y = np.array(response),
            outer_seed = 42, 
            inner_seed = 24,
            model_dir = model_dir,
            preds_dir = preds_dir,
            names = names,
            which_index = index,
            repeats = 2,
            baseline = baseline
        )

    if verbose:
        test = pd.read_csv(preds_dir+"predictions_"+str(index)+"_False.csv")
        print(test[['preds','truth']].corr().unstack()[1]) # 
        test = pd.read_csv(preds_dir+"predictions_"+str(index)+"_True.csv")
        print(test[['preds','truth']].corr().unstack()[1]) # .groupby('repeat')
    if not not example:
        break

In [ ]:
print("emt_pb: done modelling "+cancertypes[which_run]+".")

In [ ]:
#np.array(resp.iloc[:,2])
#np.array(EMTscores.iloc[:,2])